# Mina Network Crawler

The `mina advanced node-status` command collects the status output of a remote daemon and the output can be used to scrape the network for peers. 

First, the spider is seeded with the peers from the root daemon we are spidering from. Then, it preforms a breadth-first search, attempting to get a successful telemetry output from each peer. 

Interesting Metrics to Think About: 
- geographical distribution of online block producers ✅
- online stake by geographic region 
- online stake by ip org 
- peers running with the same libp2p key 
- peers running with the same block producer key 

Spider algorithm: 
- Query status for peers known by the root spider node, output list of multiaddrs to feed the spider
- For each peer: 
    - scrape the current peer's telemetry  
    - save the peer we are currently examining
    - recursively scrape remote peer's peers up to `max_depth`
    - combine new peer lists and already observed peer lists (dedup!)
    - `return` the combined lists
    
Telemetry Keys: 

`dict_keys(['node_ip_addr', 'node_peer_id', 'sync_status', 'peers', 'block_producers', 'protocol_state_hash', 'ban_statuses', 'k_block_hashes_and_timestamps', 'git_commit', 'uptime_minutes'])`

Peer Keys:
`dict_keys(['host', 'peer_id', 'libp2p_port'])`

Known Issues: 

node-status RPC times out due to short timeout: `Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #53414 failed: "context deadline exceeded"'}`


In [1]:
import itertools
import subprocess 
import json 
from datetime import datetime 

def get_root_telemetry(pod_name="sushi-seed-847fbbcd7c-gxhhf"):
    command = f"kubectl exec {pod_name} -- mina advanced node-status --daemon-peers"
    print(f"Running command: {command}")
    get_telemetry = subprocess.run(command.split(), stdout=subprocess.PIPE, text=True)
    #print(json.loads(get_telemetry.stdout.split("\n")[1:][0]))
    # skip non-json first line of output
    if "NotFound" in get_telemetry.stdout:
        raise Exception("Error accessing pod, check the name of the pod or cluster!")
    stdout_raw = get_telemetry.stdout.split("\n")[1:]
    payload = []
    for entry in stdout_raw:
        try:
            payload.append(json.loads(entry))
        except json.JSONDecodeError:
            print(entry)
            continue
    print(f"Length of telemetry payload: {len(payload)}")
    return payload

def get_telemetry(multiaddr, pod_name="sushi-seed-847fbbcd7c-gxhhf"):
    command = f"kubectl exec {pod_name} -- mina advanced node-status --show-errors --peers {multiaddr}"
    print(f"Running command: {command}")
    get_telemetry = subprocess.run(command.split(), stdout=subprocess.PIPE, text=True,)
    # skip non-json first line of output
    try:
        stdout_raw = get_telemetry.stdout.split("\n")
    except IndexError:
        # We didnt get a response
        print(f"Error, didnt't get a parsable response.")
        print(f"stdout: {get_telemetry.stdout}, stderr: {get_telemetry.stderr}")
        print(get_telemetry)
        return None
    try:
        telemetry_response = json.loads(get_telemetry.stdout)
        
        if "error" in telemetry_response:
            print(f"Error in telemetry call: {telemetry_response}")
            return telemetry_response
        else:
            return telemetry_response
    except json.JSONDecodeError:
        print(f"Error, didnt't get a parsable response. Instead got: {stdout_raw}")
        print(f"stdout: {get_telemetry.stdout}, stderr: {get_telemetry.stderr}")
        return None

In [23]:
root_telemetry = get_root_telemetry(pod_name="sushi-seed-847fbbcd7c-gxhhf")

peers = list(map(lambda t: t["peers"], root_telemetry))
flat_list = list(itertools.chain(*peers))
unique_peers = [dict(t) for t in {tuple(d.items()) for d in flat_list}]
multiaddrs = list(map(lambda peer: f"/ip4/{peer['host']}/tcp/{peer['libp2p_port']}/p2p/{peer['peer_id']}", unique_peers))

print(len(flat_list))
print(len(set(multiaddrs)))

Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --daemon-peers

Length of telemetry payload: 4
244
213


In [3]:
observed_peers = []
for peer in multiaddrs:
    telemetry = get_telemetry(peer)
    payload = {
        "telemetry": telemetry,
        "multiaddr": peer,
        "info": {}
    }
    if telemetry == None:
        print("No Response :(")
    observed_peers.append(payload)

Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/185.67.3.103/tcp/8302/p2p/12D3KooWPMY9h2wN6uxckoRNyP9pNjkMg1ANBKLYYwK36qdZvXNR
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58441 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/62.171.133.104/tcp/8302/p2p/12D3KooWFBHKuquNwDVJZ76bxg1niDQH6fv787Qmpy9TLAAKUpHX
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58442 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/188.34.183.17/tcp/8302/p2p/12D3KooWFCDGWs9eaHd3iGirj2ForY4CGbjAiTi6VqSFQs8jt5yW
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #5

Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58472 failed: "failed to dial 12D3KooWH2akF49EvajRhymjeQPop4guajGBmjV8cadXzHk7Ldj1: all dials failed\\n  * [/ip4/31.20.7.76/tcp/8302] failed to negotiate security protocol: peer id mismatch: expected 12D3KooWH2akF49EvajRhymjeQPop4guajGBmjV8cadXzHk7Ldj1, but remote key matches 12D3KooWA1HfP3j67EX91VbRGgJwJYu3QfwKpS6sWXKuKmixJbES"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/144.76.145.150/tcp/8302/p2p/12D3KooWMNeysqcnNsXNb5iTL3AFapjmJmpxyVb6hh4mpLHhvBCC
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58474 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/136.243.154.215/tcp/8302/p2p/12D3KooWKU3dsHQaJe2ATa22xxcKESTmjeKtEBvVPYEpfC4QXdmT
Error in tel

Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58501 failed: "failed to dial 12D3KooWDkp86jLkDEJWaLvwRD6JS33hW9Q7K1vbgdU1tAdj6ySM: all dials failed\\n  * [/ip4/52.11.186.247/tcp/8302] failed to negotiate security protocol: peer id mismatch: expected 12D3KooWDkp86jLkDEJWaLvwRD6JS33hW9Q7K1vbgdU1tAdj6ySM, but remote key matches 12D3KooWBov1SERpqMJpZZD2Pq3aW8hJZY5GXnRnMT5W4BACAoS9"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/157.90.130.253/tcp/8302/p2p/12D3KooWM9Y3TSZZT5mmbeVyzFb5NrjEqMeTXTEk9n5HfSjDkhZN
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58503 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/51.222.11.196/tcp/8302/p2p/12D3KooWPX1ymnhSNyZCve8UttJzW1wfB4QUp6mxs7uC1wWWN67a
Error in te

Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58533 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/54.36.121.173/tcp/8302/p2p/12D3KooWH9CPa9dU7WBs64jyM5LGkTa8TPonUj9r8hPeGR8cVC1V
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58534 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/95.217.42.20/tcp/8302/p2p/12D3KooWDnLTMJwHDk2cMwQY6QPYxsJvGg4sjzZE5eUBSCi6Z7wT
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58535 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/212.186.73.73/tcp/8302/p2p/12D3KooWHD

Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58562 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/161.97.79.248/tcp/8302/p2p/12D3KooWEdw4HLALcTXJDppu7rDdQKBDFuq5UxgF6cR2uLm5Mgou
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58563 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/65.21.58.217/tcp/8302/p2p/12D3KooWGAoqkiomFu21Qp2Tps6SXnmMTm7Hfg189qQad7kvq6e9
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58564 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/157.90.32.148/tcp/10108/p2p/12D3KooWE

Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58599 failed: "failed to dial 12D3KooWRpBA9UHdNj3QayH1guaYMxEV5gkByUp3fth1YhKJQ7ou: all dials failed\\n  * [/ip4/80.241.246.226/tcp/34194] dial tcp4 80.241.246.226:34194: connect: connection refused"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/35.238.97.69/tcp/8302/p2p/12D3KooWKCDbhjxGfrsNJVxFvaAgK9TpBfZVdBRDPDu5W9UGhvpd
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58602 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/176.9.94.138/tcp/8302/p2p/12D3KooWP7gVe3JhpBywSHZUTEUUMQxMHDTc85J7aX5hh3b6xjjE
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58603 failed: "context deadline exceede

Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58626 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/80.241.246.226/tcp/8302/p2p/12D3KooWRpBA9UHdNj3QayH1guaYMxEV5gkByUp3fth1YhKJQ7ou
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58627 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/168.119.212.145/tcp/8302/p2p/12D3KooWEstyrM8yeVbx7TDFQPgfGe7kN74ZL9skom2sYvnmv53N
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58628 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/18.185.174.225/tcp/8302/p2p/12D3K

Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58657 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/95.216.146.137/tcp/8302/p2p/12D3KooWAkB9WSsmYpHq5nB4CnVdpa7zKP9t8MYnH1tFZ1yJCumu
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58658 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/82.69.152.196/tcp/8302/p2p/12D3KooWPPXxH9uSSGW4nhXMzxz6sY63TH6YfgFLNXSD2EnrJDWN
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58659 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/47.91.18.226/tcp/8302/p2p/12D3KooWE

In [4]:
# Unpack Peers from Telemetry Responses
for index, peer in enumerate(observed_peers): 
    if peer["telemetry"] and "peers" in peer["telemetry"]:
        for peer in peer["telemetry"]["peers"]:
            multiaddr = f"/ip4/{peer['host']}/tcp/{peer['libp2p_port']}/p2p/{peer['peer_id']}"
            if not next((item for item in observed_peers if item["multiaddr"] == multiaddr), None): 
                payload = {
                    "telemetry": {},
                    "multiaddr": multiaddr,
                    "info": {}
                }
                observed_peers.append(payload)

In [5]:
# Update existing List
for index, peer in enumerate(observed_peers):
    if peer["telemetry"] == None or "error" in peer["telemetry"]:
        telemetry = get_telemetry(peer["multiaddr"])
        peer["telemetry"] = telemetry
        observed_peers[index] = peer


Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/185.67.3.103/tcp/8302/p2p/12D3KooWPMY9h2wN6uxckoRNyP9pNjkMg1ANBKLYYwK36qdZvXNR
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58683 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/62.171.133.104/tcp/8302/p2p/12D3KooWFBHKuquNwDVJZ76bxg1niDQH6fv787Qmpy9TLAAKUpHX
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58685 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/188.34.183.17/tcp/8302/p2p/12D3KooWFCDGWs9eaHd3iGirj2ForY4CGbjAiTi6VqSFQs8jt5yW
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #5

Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58713 failed: "failed to dial 12D3KooWH2akF49EvajRhymjeQPop4guajGBmjV8cadXzHk7Ldj1: all dials failed\\n  * [/ip4/31.20.7.76/tcp/8302] failed to negotiate security protocol: peer id mismatch: expected 12D3KooWH2akF49EvajRhymjeQPop4guajGBmjV8cadXzHk7Ldj1, but remote key matches 12D3KooWA1HfP3j67EX91VbRGgJwJYu3QfwKpS6sWXKuKmixJbES"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/144.76.145.150/tcp/8302/p2p/12D3KooWMNeysqcnNsXNb5iTL3AFapjmJmpxyVb6hh4mpLHhvBCC
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58714 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/136.243.154.215/tcp/8302/p2p/12D3KooWKU3dsHQaJe2ATa22xxcKESTmjeKtEBvVPYEpfC4QXdmT
Error in tel

Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58741 failed: "failed to dial 12D3KooWDkp86jLkDEJWaLvwRD6JS33hW9Q7K1vbgdU1tAdj6ySM: all dials failed\\n  * [/ip4/52.11.186.247/tcp/8302] failed to negotiate security protocol: peer id mismatch: expected 12D3KooWDkp86jLkDEJWaLvwRD6JS33hW9Q7K1vbgdU1tAdj6ySM, but remote key matches 12D3KooWBov1SERpqMJpZZD2Pq3aW8hJZY5GXnRnMT5W4BACAoS9"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/157.90.130.253/tcp/8302/p2p/12D3KooWM9Y3TSZZT5mmbeVyzFb5NrjEqMeTXTEk9n5HfSjDkhZN
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58742 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/51.222.11.196/tcp/8302/p2p/12D3KooWPX1ymnhSNyZCve8UttJzW1wfB4QUp6mxs7uC1wWWN67a
Error in te

Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58774 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/54.36.121.173/tcp/8302/p2p/12D3KooWH9CPa9dU7WBs64jyM5LGkTa8TPonUj9r8hPeGR8cVC1V
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58775 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/95.217.42.20/tcp/8302/p2p/12D3KooWDnLTMJwHDk2cMwQY6QPYxsJvGg4sjzZE5eUBSCi6Z7wT
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58776 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/212.186.73.73/tcp/8302/p2p/12D3KooWHD

Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58805 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/161.97.79.248/tcp/8302/p2p/12D3KooWEdw4HLALcTXJDppu7rDdQKBDFuq5UxgF6cR2uLm5Mgou
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58806 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/65.21.58.217/tcp/8302/p2p/12D3KooWGAoqkiomFu21Qp2Tps6SXnmMTm7Hfg189qQad7kvq6e9
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58807 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/157.90.32.148/tcp/10108/p2p/12D3KooWE

Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58838 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/176.9.94.138/tcp/8302/p2p/12D3KooWP7gVe3JhpBywSHZUTEUUMQxMHDTc85J7aX5hh3b6xjjE
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58839 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/95.216.8.171/tcp/52168/p2p/12D3KooWGEJXUcQdEQygG5pkzKxMsiv9wh2Y5wNWXaonmDf82RYb
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58840 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/165.232.149.254/tcp/8302/p2p/12D3KooW

Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58863 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/168.119.212.145/tcp/8302/p2p/12D3KooWEstyrM8yeVbx7TDFQPgfGe7kN74ZL9skom2sYvnmv53N
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58864 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/18.185.174.225/tcp/8302/p2p/12D3KooWLRYJRi3Cq6X86witS3uqpNg6D2SwGt3MFxCdYAAqFT78
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58865 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/45.32.21.162/tcp/8302/p2p/12D3Koo

Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58893 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/47.91.18.226/tcp/8302/p2p/12D3KooWEzS2YrWJ5n35ovVYiWLTB2WUHcnWdEt6ZRbbKg99UgcP
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58894 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/144.126.222.99/tcp/8302/p2p/12D3KooWQmveTY26nu9HqXrGXpAmpXJeTrMdR5GAHkSHy27mmyWV
Error in telemetry call: {'error': {'commit_id': '09758671a488c6bd4c8f207524d3c962fdcd5d75', 'string': 'RPC #58895 failed: "context deadline exceeded"'}}
Running command: kubectl exec sushi-seed-847fbbcd7c-gxhhf -- mina advanced node-status --show-errors --peers /ip4/209.126.9.140/tcp/8302/p2p/12D3KooWG

In [6]:
error_peers = [k['telemetry'] for k in observed_peers if k["telemetry"].get('error')]
print(f"Error Peers: {len(error_peers)}")
print(f"Observed Peers: {len(observed_peers)}")

Error Peers: 181
Observed Peers: 433


In [7]:
current_timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
filename = f"peer-network-scrape-{current_timestamp}.json"
with open(filename, "w") as file:
    file.write(json.dumps(list(observed_peers)))

In [8]:
print(f"{len(observed_peers)} peers observed")
next((item for item in observed_peers if "error" not in item["telemetry"]), {})["telemetry"].keys()

433 peers observed


dict_keys(['node_ip_addr', 'node_peer_id', 'sync_status', 'peers', 'block_producers', 'protocol_state_hash', 'ban_statuses', 'k_block_hashes_and_timestamps', 'git_commit', 'uptime_minutes'])

In [9]:
import ipinfo
import os
access_token = os.getenv("IPINFO_TOKEN") 
handler = ipinfo.getHandler(access_token)

locations = []
for peer in observed_peers:
    multiaddr_parts = peer["multiaddr"].split("/")
    ip_address = multiaddr_parts[2]
    details = handler.getDetails(ip_address).all
    
    try:
        city = details["city"]
    except KeyError: 
        city = "None"
        
    try:
        country = details["country"]
    except KeyError: 
        country = "None"
        
    try:
        org = details["org"]
    except KeyError: 
        org = "None"
    
    try:
        lat = details["latitude"]
    except KeyError: 
        lat = 0
        
    try:
        long = details["longitude"]
    except KeyError: 
        long = 0
        
        
        
    peer["info"] = {
        "lat": lat, 
        "long": long, 
        "multiaddr": peer["multiaddr"], 
        "peer_id": multiaddr_parts[6], 
        "ip_address": ip_address, 
        "city": city, 
        "country": country, 
        "org": org
    }
    
    

In [10]:
import pandas as pd


df = pd.DataFrame(observed_peers)

In [11]:
# https://stackoverflow.com/questions/39899005/how-to-flatten-a-pandas-dataframe-with-some-columns-as-json
import ast
from pandas import json_normalize

def only_dict(d):
    '''
    Convert json string representation of dictionary to a python dict
    '''
    return ast.literal_eval(d)

info_flat = json_normalize(df['info'].tolist()).add_prefix('info.')
info_flat
merged = pd.merge(df, info_flat, left_on='multiaddr', right_on='info.multiaddr')
merged

,telemetry,multiaddr,info,info.lat,info.long,info.multiaddr,info.peer_id,info.ip_address,info.city,info.country,info.org
0,{'error': {'commit_id': '09758671a488c6bd4c8f2...,/ip4/185.67.3.103/tcp/8302/p2p/12D3KooWPMY9h2w...,"{'lat': '50.4547', 'long': '30.5238', 'multiad...",50.4547,30.5238,/ip4/185.67.3.103/tcp/8302/p2p/12D3KooWPMY9h2w...,12D3KooWPMY9h2wN6uxckoRNyP9pNjkMg1ANBKLYYwK36q...,185.67.3.103,Kyiv,UA,AS196645 LTD HOSTPRO LAB
1,{'error': {'commit_id': '09758671a488c6bd4c8f2...,/ip4/62.171.133.104/tcp/8302/p2p/12D3KooWFBHKu...,"{'lat': '47.4921', 'long': '11.0958', 'multiad...",47.4921,11.0958,/ip4/62.171.133.104/tcp/8302/p2p/12D3KooWFBHKu...,12D3KooWFBHKuquNwDVJZ76bxg1niDQH6fv787Qmpy9TLA...,62.171.133.104,Garmisch-Partenkirchen,DE,AS51167 Contabo GmbH
2,{'error': {'commit_id': '09758671a488c6bd4c8f2...,/ip4/188.34.183.17/tcp/8302/p2p/12D3KooWFCDGWs...,"{'lat': '50.1155', 'long': '8.6842', 'multiadd...",50.1155,8.6842,/ip4/188.34.183.17/tcp/8302/p2p/12D3KooWFCDGWs...,12D3KooWFCDGWs9eaHd3iGirj2ForY4CGbjAiTi6VqSFQs...,188.34.183.17,Frankfurt am Main,DE,AS24940 Hetzner Online GmbH
3,{'error': {'commit_id': '09758671a488c6bd4c8f2...,/ip4/35.226.250.48/tcp/8302/p2p/12D3KooWMvU6gQ...,"{'lat': '41.2619', 'long': '-95.8608', 'multia...",41.2619,-95.8608,/ip4/35.226.250.48/tcp/8302/p2p/12D3KooWMvU6gQ...,12D3KooWMvU6gQyz272PmRnEvxQpFARHCszm7phA1LXjQU...,35.226.250.48,Council Bluffs,US,AS15169 Google LLC
4,{'error': {'commit_id': '09758671a488c6bd4c8f2...,/ip4/51.222.11.200/tcp/8302/p2p/12D3KooWEEjqKm...,"{'lat': '45.5088', 'long': '-73.5878', 'multia...",45.5088,-73.5878,/ip4/51.222.11.200/tcp/8302/p2p/12D3KooWEEjqKm...,12D3KooWEEjqKmkdwFJLKpcokjMkWhNXVaFwykqy4Adctf...,51.222.11.200,Montréal,CA,AS16276 OVH SAS
...,...,...,...,...,...,...,...,...,...,...,...
428,{},/ip4/176.106.59.120/tcp/11433/p2p/12D3KooWKoEJ...,"{'lat': '56.9460', 'long': '24.1059', 'multiad...",56.9460,24.1059,/ip4/176.106.59.120/tcp/11433/p2p/12D3KooWKoEJ...,12D3KooWKoEJWQPNpD8Z2yA66c8NZGRGPTNy57rM7Z6HQ3...,176.106.59.120,Riga,LV,AS24589 Telenet SIA
429,{},/ip4/134.195.196.56/tcp/8302/p2p/12D3KooWPdLJt...,"{'lat': '45.5088', 'long': '-73.5878', 'multia...",45.5088,-73.5878,/ip4/134.195.196.56/tcp/8302/p2p/12D3KooWPdLJt...,12D3KooWPdLJtpw2toshJSynW9HxmCjCmGppmtf7gnxLYa...,134.195.196.56,Montréal,CA,AS62563 GLOBALTELEHOST Corp.
430,{},/ip4/65.21.5.88/tcp/8302/p2p/12D3KooWJoXMxVHrx...,"{'lat': '60.3540', 'long': '24.9794', 'multiad...",60.3540,24.9794,/ip4/65.21.5.88/tcp/8302/p2p/12D3KooWJoXMxVHrx...,12D3KooWJoXMxVHrxLydtshAD4zGNjnymoccYikqSzB7H3...,65.21.5.88,Tuusula,FI,AS24940 Hetzner Online GmbH
431,{},/ip4/95.217.122.162/tcp/1024/p2p/12D3KooWGmqyj...,"{'lat': '60.3540', 'long': '24.9794', 'multiad...",60.3540,24.9794,/ip4/95.217.122.162/tcp/1024/p2p/12D3KooWGmqyj...,12D3KooWGmqyj1hodJS5hE4NfRZo26GcjRcYiPS7A9XSJ9...,95.217.122.162,Tuusula,FI,AS24940 Hetzner Online GmbH


In [82]:
# import country_converter as coco
# country_counts = merged["info.country"].value_counts()


# def remap_country_dict(input_dict):    
#     cc = coco.CountryConverter()
#     output_dict = {}
#     for key in input_dict.keys():
#         result = cc.convert(names=key, to='ISO3')
#         output_dict[result] = input_dict[key]
#     return output_dict

# remapped = remap_country_dict(country_counts)
# remapped

In [14]:
import country_converter as coco

cc = coco.CountryConverter()
intermediate_df = merged
for key in merged["info.country"].unique():
    new_key = cc.convert(names=key, to='ISO3')
    intermediate_df = intermediate_df.replace(to_replace=key, value=new_key)
country_counts = intermediate_df["info.country"].value_counts()

country_counts

/usr/local/lib/python3.9/site-packages/country_converter/country_converter.py:578: FutureWarning: The default value of regex will change from True to False in a future version.
  _match_col = self.data[src_format].astype(str).str.replace("\\..*", "")
None not found in regex


USA          114
DEU           98
FIN           77
RUS           24
CAN           20
CHN           16
FRA           13
NLD           10
UKR            8
JPN            7
GBR            7
ESP            6
SGP            5
HKG            4
LTU            3
not found      3
IND            2
VNM            2
GEO            2
TUR            2
AUT            2
ITA            1
KOR            1
ROU            1
BEL            1
IRL            1
LVA            1
PHL            1
THA            1
Name: info.country, dtype: int64

In [15]:
unique_countries = intermediate_df["info.country"].unique()

In [16]:
output = []
for country in unique_countries: 
    row = {
        "country": country, 
        "count": country_counts[country]
    }
    output.append(row)
country_count_df = pd.DataFrame(output)
country_count_df

,country,count
0,UKR,8
1,DEU,98
2,USA,114
3,CAN,20
4,RUS,24
5,NLD,10
6,GBR,7
7,FIN,77
8,FRA,13
9,CHN,16


In [20]:
import plotly.express as px
fig = px.scatter_geo(
    country_count_df, 
    locations="country",    
    size="count",                
    projection="natural earth",
    title="Count of Nodes per IP-Country")
fig.show()


### Example Output:
<img src="images/node-count-example.png">

In [22]:
intermediate_df["info.org"].value_counts().head(40)

AS24940 Hetzner Online GmbH                                       144
AS14061 DigitalOcean, LLC                                          34
AS15169 Google LLC                                                 34
AS16509 Amazon.com, Inc.                                           18
AS16276 OVH SAS                                                    16
AS40021 Contabo Inc.                                               12
AS51167 Contabo GmbH                                               12
AS20473 The Constant Company, LLC                                  10
AS45102 Alibaba (US) Technology Co., Ltd.                           8
AS21409 Ikoula Net SAS                                              7
AS14618 Amazon.com, Inc.                                            6
AS4837 CHINA UNICOM China169 Backbone                               5
AS25513 PJSC Moscow city telephone network                          4
AS7922 Comcast Cable Communications, LLC                            4
AS63023 GTHost      

In [29]:
import plotly.express as px
fig = px.scatter_geo(
    intermediate_df, 
    lat="info.lat",    
    lon="info.long",                
    projection="natural earth",
    color="info.country",
    title="IP Location of Observed Nodes")
fig.show()

### Example Output:
<img src="images/node-ip-location.png">